# Adding a user-defined decision module to the turtles: The boids model

The [boids](https://en.wikipedia.org/wiki/Boids) (bird-oid) model has been invented by [Craig Reynolds](https://en.wikipedia.org/wiki/Craig_Reynolds_(computer_graphics)) in 1986 to simulate the flocking behavior of birds. It is based on 3 principles:
    
* separation: boids tend to avoid other boids that are too close,

* alignment: boids tend to align their velocity to boids that are not too close and not too far away,

* cohesion: bois tend to move towards boids that are too far away.

While these rules are essentially heuristic, they can be implemented defining three areas (repulsion, orientation, attraction) for each principle. 

* Boids change their orientation to get away from other boids in the repulsion area,

* Boids change their orientation and speed to match those of other boids in the orientation area,

* Boids change their orientation to get to other boids in the attraction area.

## Grab dependencies

In [3]:
#!python
#@repository("~/.m2/repository/")
#@dependency(group="fr.univ_artois.lgi2a", module="similar2logo-lib", version="1.0-SNAPSHOT")

import math

from fr.univ_artois.lgi2a.similar.extendedkernel.libs.abstractimpl import AbstractAgtDecisionModel
from fr.univ_artois.lgi2a.similar.microkernel import AgentCategory
from fr.univ_artois.lgi2a.similar.microkernel.ISimulationModel import AgentInitializationData
from fr.univ_artois.lgi2a.similar2logo.kernel.initializations import AbstractLogoSimulationModel
from fr.univ_artois.lgi2a.similar2logo.kernel.model import LogoSimulationParameters
from fr.univ_artois.lgi2a.similar2logo.kernel.model.agents.turtle import TurtleFactory, \
    TurtleAgentCategory
from fr.univ_artois.lgi2a.similar2logo.kernel.model.influences import ChangeDirection, \
    ChangeSpeed
from fr.univ_artois.lgi2a.similar2logo.kernel.model.levels import LogoSimulationLevelList
from fr.univ_artois.lgi2a.similar2logo.kernel.tools import MathUtil
from fr.univ_artois.lgi2a.similar2logo.lib.model import ConeBasedPerceptionModel
from fr.univ_artois.lgi2a.similar2logo.lib.tools.html import Similar2LogoHtmlRunner
from fr.univ_artois.lgi2a.similar2logo.lib.tools.math import MeanAngle
from fr.univ_artois.lgi2a.similar2logo.lib.tools.random import PRNG

No Outputs

## Model parameters

The model parameters and their default values are defined.

In [4]:
class BoidsSimulationParameters(LogoSimulationParameters): 

    def __init__(self):
        self.repulsionDistance = 1.0
        self.orientationDistance = 2.0
        self.attractionDistance = 4.0
        self.repulsionWeight = 10.0
        self.orientationWeight = 20.0
        self.attractionWeight = 0.1
        self.maxInitialSpeed = 2.0
        self.minInitialSpeed = 1.0
        self.perceptionAngle = math.pi
        self.nbOfAgents = 200
        self.maxAngle = math.pi / 4

No Outputs

## Decision model

The decision model consists in changing the direction and speed of the boids according to the previously described rules.
To define a decision model, the modeler must define an object that extends `AbstractAgtDecisionModel` and implement the `decide` method.

In [5]:
class BoidDecisionModel(AbstractAgtDecisionModel):
    
    def __init__(self, parameters):
        self.parameters = parameters
        super(BoidDecisionModel, self).__init__(LogoSimulationLevelList.LOGO)
    
    def decide(
        self,
        timeLowerBound,
        timeUpperBound,
        globalState,
        publicLocalState,
        privateLocalState,
        perceivedData,
        producedInfluences
    ):
        if not perceivedData.turtles.isEmpty():
            orientationSpeed = 0.0
            nbOfTurtlesInOrientationArea = 0
            meanAngle = MeanAngle()
            for perceivedTurtle in perceivedData.turtles:
                if perceivedTurtle.distanceTo <= self.parameters.repulsionDistance:
                    meanAngle.add(
                        publicLocalState.direction - perceivedTurtle.directionTo,
                        self.parameters.repulsionWeight
                    )
                elif perceivedTurtle.distanceTo <= self.parameters.orientationDistance:
                    meanAngle.add(
                        perceivedTurtle.content.direction - publicLocalState.direction,
                        self.parameters.orientationWeight
                    )
                    orientationSpeed += perceivedTurtle.content.speed - publicLocalState.speed
                    nbOfTurtlesInOrientationArea+=1
                elif perceivedTurtle.distanceTo <= self.parameters.attractionDistance:
                    meanAngle.add(
                        perceivedTurtle.directionTo - publicLocalState.direction,
                        self.parameters.attractionWeight
                    )
   
            dd = meanAngle.value()
            if not MathUtil.areEqual(dd, 0.0):
                if dd > self.parameters.maxAngle:
                    dd = self.parameters.maxAngle
                elif dd < -self.parameters.maxAngle:
                    dd = -self.parameters.maxAngle

                producedInfluences.add(
                    ChangeDirection(
                        timeLowerBound,
                        timeUpperBound,
                        dd,
                        publicLocalState
                    )
                )
                
            if  nbOfTurtlesInOrientationArea > 0:
                orientationSpeed /= nbOfTurtlesInOrientationArea;
                producedInfluences.add(
                    ChangeSpeed(
                        timeLowerBound,
                        timeUpperBound,
                        orientationSpeed,
                        publicLocalState
                    )
                )

No Outputs

## The simulation model

In the simulation model defined in our example, boids are randomly located in the environment with a random orientation and speed.

In [6]:
class BoidsSimulationModel(AbstractLogoSimulationModel):
    
    def __init__(self, parameters):
        super(BoidsSimulationModel, self).__init__(parameters)
    
    def generateBoid(self, p):
        return TurtleFactory.generate(
            ConeBasedPerceptionModel(
                p.attractionDistance, p.perceptionAngle, True, False, False
            ),
            BoidDecisionModel(p),
            AgentCategory("b", [TurtleAgentCategory.CATEGORY]),
            PRNG.get().randomAngle(),
            p.minInitialSpeed + PRNG.get().randomDouble() * (
                p.maxInitialSpeed - p.minInitialSpeed
            ),
            0.0,
            PRNG.get().randomDouble() * p.gridWidth,
            PRNG.get().randomDouble() * p.gridHeight
        )
        
    def generateAgents(self, parameters, levels):
        result = AgentInitializationData()
        for i in range(0, parameters.nbOfAgents):
            result.agents.add(self.generateBoid(parameters))
        return result

No Outputs

## Launch the HTML runner

Finally, we launch and configure the HTML runner.

In [7]:
runner = Similar2LogoHtmlRunner()
runner.config.setExportAgents(True)
model = BoidsSimulationModel(BoidsSimulationParameters())
runner.initializeRunner(model)
runner.showView()

No Outputs